# W1M3 - ETL 프로세스 구현하기

### 요구사항
IMF에서 제공하는 국가별 GDP 데이터 ETL 프로세스

In [1]:
# ETL 프로세스를 구현한 etl_project_gdp.py 불러오기
import etl_project_gdp as epg

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Extract
epg.extract_gdp('raw_data_gdp.json')

Extract 단계는 raw data를 추출하는 단계임으로, 웹 스크래핑을 진행하여 가공되지 않은 테이블을 가져오고 json으로 파일 형식을 바꾸어 저장하는 과정으로 정의.

미션 시나리오를 생각했을 때, 해외 사업 확장을 위해서는 테이블의 다른 정보들도 필요할 수 있을 것이라고 예상되고, 테이블 갱신 전 과거 데이터도 필요할 것이라고 예상되어, Extract 함수에서 파일명을 인자로 받아, 여러 raw data 파일로 관리할 수 있도록 구현하였다.

추가적인 리뷰를 듣고 생각을 더해봤을 때, beautifulsoup을 이용해서 html을 jupyter 환경에서 읽는 것은 그렇지 않으면, Extract 과정이 실행되지 않기 때문에 무조건적으로 필요한 과정이다. 그렇지만 이를 기능요구사항에 맞게 바로 json으로 저장하지 않고, 먼저 데이터 프레임으로 바꾸고 다시 json으로 바꾸는 것은 목적을 가지고 바꾼 것이기 때문에 완전한 의미의 raw data는 아니었다는 생각이 든다. 하지만 이번 미션에서는 html을 json으로 바로 변화시키는 것이 둘을 맵핑하는 전처리 과정과 반복문을 이용하기에 판다스를 이용해서 저장을 하는 것이 비용적으로 이득이라고 판단하였다.

In [5]:
# Transform
epg.transform1_gdp('raw_data_gdp.json')
epg.transform2_gdp()
epg.transform3_gdp()
df_gdp_final = epg.transform4_gdp()
df_gdp_final

,country,gdp,continent
1,United States,30337.16,NA
2,China,19534.89,AS
3,Germany,4921.56,EU
4,Japan,4389.33,AS
5,India,4271.92,AS
...,...,...,...
179,Greenland,NaN,NA
185,Eritrea,NaN,AF
186,Zanzibar,NaN,AF
193,Sint Maarten,NaN,EU


리뷰를 듣기 전 Transform 단계를 세분화하지 않고 묶어서 함수화하였는데, 이는 대용량의 데이터를 변환하는 과정에서 오류가 발생하면 시간적인 비용 소모가 크다는 단점이 생기게 된다. 그렇기에 방대한 raw data가 있을 때는 Transpose 단계를 세분화하여 나누는 것이 필요할 것이다. 로그 기록을 확인하고 각 단계별 파일을 저장하여, 오류가 발생했을 때 Transpose 과정의 처음으로 돌아가지 않게 안전 장치를 해야한다.

이를 바탕으로 이번 미션에서는 Transpose 과정을 4단계로 나누어서 오류를 대비하였다. 그리고 Transpose 단계마다 파일을 json 파일로 저장하도록 구현하였다. Extract 과정과 다르게 Transpose 파일은 변환 과정 중 오류 발생 대비 장치로 생각하여 'transform_gdp.json'을 사용해서 하나의 파일로 관리하도록 하였다. 마지막 Transpose 단계에서 데이터 프레임을 반환한 이유는 Load 과정에 이어 설명하겠다.

In [ ]:
# Load
epg.load_gdp(df_gdp_final, 'load_data_gdp.json')

리뷰 전에는 Load 단계를 포함하지 않고, 바로 출력을 할 수 있도록 구현했다. 물론, GDP 데이터처럼 작은 데이터를 적재할 필요는 없지만, 이 부분에 대한 고려없이 적재 과정을 생략하는 것은 적절하지 않았다고 생각한다. 그래서 데이터의 크기를 고려하고, ETL 프로세스 관련 미션이라는 특성을 고려했을 때, Load 과정을 단순히 json으로써 저장하는 것으로 정의하였다.

이렇게 정의하고 보니, load 단계는 마지막 Transform 단계에서 생성되는 데이터 프레임을 json 저장하는 것과 동일하다. ETL 단계를 명확히 구분하고, 파일을 불러오는 과정을 줄이기 위해, Transform 마지막 단계는 json 저장을 하지 않고, 데이터 프레임을 반환하는 것으로 처리하였다.

Load 데이터도 Raw data와 마찬가지로 시나리오상 과거 데이터가 필요할 것으로 예상되어, load data를 파일명으로써 관리할 수 있도록 구현하였다.

In [7]:
# Display
epg.display('load_data_gdp.json')

          country       gdp continent
1   United States  30337.16        NA
2           China  19534.89        AS
3         Germany   4921.56        EU
4           Japan   4389.33        AS
5           India   4271.92        AS
..            ...       ...       ...
69          Kenya    116.32        AF
70         Angola    113.29        AF
71      Guatemala    112.37        NA
72           Oman    110.99        AS
73      Venezuela    106.33        SA

[72 rows x 3 columns]
                gdp
continent          
NA         6946.500
AS         6327.178
EU         3318.112
SA          797.566
OC          436.658
AF          298.422


In [9]:
# ETL process
epg.ETL_gdp('raw_data_gdp.json', 'load_data_gdp.json')

          country       gdp continent
1   United States  30337.16        NA
2           China  19534.89        AS
3         Germany   4921.56        EU
4           Japan   4389.33        AS
5           India   4271.92        AS
..            ...       ...       ...
69          Kenya    116.32        AF
70         Angola    113.29        AF
71      Guatemala    112.37        NA
72           Oman    110.99        AS
73      Venezuela    106.33        SA

[72 rows x 3 columns]
                gdp
continent          
NA         6946.500
AS         6327.178
EU         3318.112
SA          797.566
OC          436.658
AF          298.422


### 추가 요구 사항
데이터 베이스 저장 후 SQL을 이용한 화면 출력.

In [10]:
# 추가 요구 사항을 반영하여 ETL 프로세스를 구현한 etl_project_gdp_with_sql.py 불러오기
import etl_project_gdp_with_sql as epgs

In [12]:
# Extract
epgs.extract_gdp('raw_data_gdp.json')

In [13]:
# Transform
epgs.transform1_gdp('raw_data_gdp.json')
epgs.transform2_gdp()
epgs.transform3_gdp()
epgs.transform4_gdp()

앞선 ETL 프로세스의 Transform 마지막 단계와 달리 load 방식이 변화되었기에, 최종 테이블을 json 파일로 저장하도록 변경하였다.

In [14]:
# Load
epgs.load_gdp()

앞선 ETL 프로세스의 Load 과정과 달리, 이번 적재 과정은 World_Economies.db 데이터 베이스에 json 파일을 테이블로 적재하는 과정으로 변경하였다.

In [15]:
# Display
epgs.display()

          Country  GDP_USD_billion Continent
0   United States         30337.16        NA
1           China         19534.89        AS
2         Germany          4921.56        EU
3           Japan          4389.33        AS
4           India          4271.92        AS
..            ...              ...       ...
67          Kenya           116.32        AF
68         Angola           113.29        AF
69      Guatemala           112.37        NA
70           Oman           110.99        AS
71      Venezuela           106.33        SA

[72 rows x 3 columns]
   TOP5 AVERAGE Continent
0      6946.500        NA
1      6327.178        AS
2      3318.112        EU
3       797.566        SA
4       436.658        OC
5       298.422        AF


In [17]:
# ETL process
epgs.ETL_gdp('raw_data_gdp.json')

          Country  GDP_USD_billion Continent
0   United States         30337.16        NA
1           China         19534.89        AS
2         Germany          4921.56        EU
3           Japan          4389.33        AS
4           India          4271.92        AS
..            ...              ...       ...
67          Kenya           116.32        AF
68         Angola           113.29        AF
69      Guatemala           112.37        NA
70           Oman           110.99        AS
71      Venezuela           106.33        SA

[72 rows x 3 columns]
   TOP5 AVERAGE Continent
0      6946.500        NA
1      6327.178        AS
2      3318.112        EU
3       797.566        SA
4       436.658        OC
5       298.422        AF


앞선 ETL 프로세스와 동일한 결과

### 팀 활동 요구사항

### Q1. wikipeida 페이지가 아닌, IMF 홈페이지에서 직접 데이터를 가져오는 방법은 없을까요? 어떻게 하면 될까요?

IMF 홈페이지를 살펴본 결과, IMF에서 제공하는 api인 'https://www.imf.org/external/datamapper/api/v1/NGDPD'에 접근하여 테이블을 가져와야 한다.</br>
또한, 이 api는 html이 아닌 json 구조의 데이터를 제공하기에 beautilfulsoup4 패키지보다는 json 패키지를 이용해서 웹 스크래핑을 시도해야 한다.</br>
실습한 결과, html 방식에 비해 계층적 구조에서의 평탄화 작업이 필요하며 from_dict를 통해 딕셔너리를 데이터프레임으로 변경해주어야 한다.

In [ ]:
import requests
import json

file = requests.get('https://www.imf.org/external/datamapper/api/v1/NGDPD', stream=True)
data = file.json()

# 데이터 평탄화 과정
ngdpd_data = data["values"]["NGDPD"]
df_hompage_gdp = pd.DataFrame.from_dict(ngdpd_data, orient="index")

# JSON 파일로 저장
df_hompage_gdp.to_json("IMF_homepage_gdp.json", indent=4)

pd.read_json("IMF_homepage_gdp.json")

성공적으로 Raw data를 생성하였다. Transpose 과정도 Raw data에 맞게 수정해주어야 할 것이다.


앞선 ETL 과정에서 국가를 지역으로 변환하는 과정을 진행했었다. 성공적으로 변환하긴 하였지만, IMF에서 구분한 국가의 지역은 6개보다 더 많게 구분지었다. 이를 수행하기 위해서는 IMF 지역에 맞는 맵핑을 진행해야 될 것이다.
또한 웹 스크래핑을 통해 직접적으로 지역별 평균 GDP를 가져오는 것은 가능했지만, 상위 5개의 국가의 평균은 웹 스크래핑 만으로는 가지고 올 수 없었기에 이 출력을 수행하기 위해서는 국가와 지역의 맵핑이 필수적이라고 결론 지었다.

### Q2. 만약 데이터가 갱신되면 과거의 데이터는 어떻게 되어야 할까요? 과거의 데이터를 조회하는 게 필요하다면 ETL 프로세스를 어떻게 변경해야 할까요?

과거 데이터가 추후에 필요한 상황이라면 갱신되기 전 과거 데이터가 저장되어 있어야한다. 과거 데이터를 저장하는 방법으로 생각한 방법은 다음과 같다.
1. Extract 과정을 진행할 때마다, raw data를 다른 파일명으로 저장한다.
2. 갱신 주기를 알고 있는 상황이라면, 로그를 확인하여 Extract 과정에서 그 주기를 초과했을 때, 새로운 파일을 저장한다.
3. 갱신이 수시로 진행된다면, 바로 이전에 추출한 raw data와 현재 추출한 raw data를 비교하여 변경점이 있는 경우에만 다른 파일로 저장한다.
4. 데이터를 데이터 베이스에 Load하며 갱신하는 과정에서 변경점이 존재할 때, 이력 테이블을 생성할 수 있도록 하는 과정을 추가한다.

방법 1은 저장 공간 측면에서는 손해이지만, 데이터 갱신 주기가 짧고 데이터 추출을 많이 하지 않을 때 유리한 방법일 것이다.</br>
방법 2는 갱신 주기가 지켜지지 않았을 때 잘못된 데이터를 추출할 수 있는 가능성은 있지만, 저장 공간 측면에서 유리한 방법일 것이다.</br>
방법 3은 raw data의 크기가 클 때 두 데이터를 비교하는 시간이 오래걸릴 수 있지만, 저장 공간 측면에서도 유리하고 과거 데이터 갱신을 무조건적으로 찾을 수 있는 방법일 것이다.</br>
방법 4는 적재하는 과정에서 과거 데이터를 관리하는 방법인데, 방법 1~3에서의 추출 조건과 결합하여 이력 테이블을 생성하는 조건으로써 활용될 수 있을 것이다.</br>